In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import librosa, librosa.display
import sklearn
import keras
from keras.models import Model, Sequential, Input
from keras.layers import Conv2D
import keras.backend as K

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5484642357825660967
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15529008455900054920
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4884790977826276738
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15855861760
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17227975246448366542
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


# Load in Data and peek at dataframe

1. 25863 mp3 mono audio files, each of 30 seconds long
2. 188 labels associated with each song

In [3]:
df = pd.read_csv("../annotations_final.csv",sep="\t")
df2=pd.read_csv("../clip_info_final.csv",sep="\t")

In [4]:
print(df.shape,df2.shape)

(25863, 190) (31382, 10)


In [5]:
df.head(10)

,clip_id,no voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,female singing,...,rap,metal,hip hop,quick,water,baroque,women,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
5,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c/lvx_nova-lvx_nova-01-contimune-30-59.mp3
6,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c/lvx_nova-lvx_nova-01-contimune-175-204.mp3
7,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c/lvx_nova-lvx_nova-01-contimune-233-262.mp3
8,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c/lvx_nova-lvx_nova-01-contimune-291-320.mp3
9,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0/american_bach_soloists-j_s__bach__cantatas_v...


In [6]:
df2.head(6)

,clip_id,track_number,title,artist,album,url,segmentStart,segmentEnd,original_url,mp3_path
0,2,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,30,59,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,146,175,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,262,291,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,291,320,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,320,349,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
5,14,1,Contimune,LVX Nova,LVX Nova,http://www.magnatune.com/artists/albums/lvxnov...,30,59,http://he3.magnatune.com/all/01--Contimune--LV...,c/lvx_nova-lvx_nova-01-contimune-30-59.mp3


# Ground truth Labels

In [7]:
y=df.values[:,1:-1]
#this y contains clip id as its first column
print(y,y.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] (25863, 188)


# Feature extraction

1. use feature extraction methods from VGGish, preprocess.py
2. perform transfer learning on VGGish network

In [8]:
#split dataset into train and test set
n=df.shape[0]
n_test=n//4
test_idx=np.arange(0,n_test)
train_idx=np.arange(n_test,n)
batch_size = 32

In [23]:
#this takes input as batch size and output the input batch and ground truth labels
def batchmatrix(batch_size,df,idx):
    #idx=np.arange(0,df.shape[0])
    np.random.shuffle(idx)
    idx_shuffled=idx[:batch_size]
    batchmatrix=np.zeros((batch_size,96,64))
    batchy=np.zeros((batch_size,188))
    for m,i in enumerate(idx_shuffled):
        x,fs=librosa.load(df["mp3_path"][i])
        X=preprocess_sound.preprocess_sound(x, fs)
        X_slice=X[random.randint(0,X.shape[0]-1),:,:]
        batchmatrix[m,:,:]=X_slice
        batchy[m,:]=df.values[i,1:-1]
    return batchmatrix,batchy

# Transfer Learning from VGGish net

In [24]:
import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim
import vggish
import preprocess_sound
import random

In [25]:
from __future__ import print_function
# from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model #save and load models
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization



In [26]:
model=vggish.VGGish(load_weights=True, weights='audioset',
           input_tensor=None, input_shape=None,
           out_dim=None, include_top=False, pooling='avg')
new_model=Sequential()
for layer in model.layers:
    new_model.add(layer)
    layer.trainable=False
    

#new_model.add(Flatten())
new_model.add(BatchNormalization())
new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation = 'relu'))
new_model.add(BatchNormalization())
new_model.add(Dropout(0.5))
new_model.add(Dense(188, activation='softmax'))

In [27]:
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 96, 64, 64)        640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 32, 128)       73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 24, 16, 128)       0         
_________________________________________________________________
conv3/conv3_1 (Conv2D)       (None, 24, 16, 256)       295168    
_________________________________________________________________
conv3/conv3_2 (Conv2D)       (None, 24, 16, 256)       590080    
_________________________________________________________________
pool3 (MaxPooling2D)         (None, 12, 8, 256)        0         
__________

# Batch processing

1. using pescador to generate streamers

In [29]:
def batchmatrix(batch_size,df,idx):
    #idx=np.arange(0,df.shape[0])
    np.random.shuffle(idx)
    idx_shuffled=idx[:batch_size]
    batchmatrix=np.zeros((batch_size,96,64,1))
    batchy=np.zeros((batch_size,188))
    for m,i in enumerate(idx_shuffled):
        x,fs=librosa.load(df["mp3_path"][i])
        X=preprocess_sound.preprocess_sound(x, fs)
        X_slice=X[random.randint(0,X.shape[0]-1),:,:]
        X_slice.reshape(-1,96,64,1)
        batchmatrix[:,m,:,:]=X_slice
        batchy[m,:]=df.values[i,1:-1]
    return batchmatrix,batchy

In [30]:
def feature_sampler(df,idx):
    #idx=np.arange(0,df.shape[0])
    np.random.shuffle(idx)
    i=idx[0]
    #feature=np.zeros((96,64))
    y=df.values[i,1:-1]
    x,fs=librosa.load(df["mp3_path"][i])
    X=preprocess_sound.preprocess_sound(x, fs)
    X_slice=X[random.randint(0,X.shape[0]-1),:,:]
    #X_slice.reshape(-1,96,64,1)
    while True:
        yield{'X': X_slice[:,:,None],'y': y}


    
def data_generator(batch_size, train_idx, active_streamers=1,
                        rate=64):
    #batch_size=32
    streamer = pescador.Streamer(feature_sampler, df,train_idx)
    #seeds.append(streamer)

    # Randomly shuffle the seeds
    #random.shuffle(seeds)

    #mux = pescador.StochasticMux(streamer, active_streamers, rate=rate)
    mux=streamer
    if batch_size == 1:
        return mux
    else:
        return pescador.maps.buffer_stream(mux, batch_size)

# Train the model

In [31]:
import pescador

In [32]:
epochs=12
batch_size=32
batches=data_generator(batch_size,train_idx,active_streamers=1,rate=64)
steps_per_epoch = len(train_idx) // batch_size

In [36]:
#create X_test and y_test
X_test,Y_test=batchmatrix(32,df,test_idx)
print(X_test.shape,Y_test.shape)

ValueError: could not broadcast input array from shape (96,64) into shape (32,64,1)

In [21]:
hist = new_model.fit_generator(
        pescador.tuples(batches, 'X', 'y'),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        verbose=1,
        validation_data=(X_test, Y_test)
        )

NameError: name 'X_test' is not defined

In [34]:
hist = new_model.fit_generator(
        pescador.maps.keras_tuples(data_generator, 'X', 'y'),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        verbose=1,
        validation_data=(X_test, Y_test)
        )

NameError: name 'X_test' is not defined

In [ ]:
scores = new_model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
new_model.fit_generator(
        pescador.tuples(batches, 'X', 'y'),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        verbose=1,
        #validation_data=(X_test, Y_test)
        )
#except KeyboardInterrupt:
#    print("Stopping early")
#finally:
#print("Finished: {}".format(datetime.datetime.now()))

scores = new_model.evaluate(X_test, Y_test, verbose=0)

# Use pickle to store preprocessed features

In [17]:
#this takes input as batch size and output the input batch and ground truth labels
def batchmatrix_pickle(df,idx, folder):
    #idx=np.arange(0,df.shape[0])
    #np.random.shuffle(idx)
    #idx_shuffled=idx[:batch_size]
    batchmatrix=np.zeros((1,96,64))
    batchy=np.zeros((batch_size,188))
    for m,i in enumerate(idx):
        x,fs=librosa.load(df["mp3_path"][i])
        X=preprocess_sound.preprocess_sound(x, fs)
        X_slice=X[random.randint(0,X.shape[0]-1),:,:]
        batchmatrix=X_slice
        batchy=df.values[i,1:-1]
        output = open('./pickles/'+folder+'/'+str(i)+'.pkl', 'wb')
        pickle.dump([batchmatrix,batchy], output)
        output.close()
        
    return 0

In [18]:
import pickle
batchmatrix_pickle(df,test_idx, test)
batchmatrix_pickle(df,train_idx, train)

0